In [661]:
import pandas as pd
import numpy as np
import io
import ast
import warnings

warnings.filterwarnings("ignore")
class moviedata():
    
    def __init__(self) -> None:
        self.credits_2_df=pd.read_csv("archive\credits-2.csv").drop_duplicates().copy()
        self.movies_df=pd.read_csv("archive\movies_metadata.csv",low_memory=False,dtype=str).filter(items=["adult","popularity","budget","genres","homepage","id","imdb_id","original_title","overview","production_companies","production_countries","release_date","spoken_languages","vote_average","vote_count"]).copy()
        # only useful remained
        #then merge the credit df with movie df
        
        self.movies_df=pd.concat([self.movies_df,self.credits_2_df],axis=1,join="outer")
        inx=self.movies_df.shape[1]-1
        self.movies_df=self.movies_df.iloc[:,[x for x in range(inx)]]
        self.movies_df=self.movies_df.dropna(axis=0,how="any")
        
        
    @staticmethod
    def search(df,**kwargs):
        newp=df.copy()
        
        for i in kwargs:
            try:
                def cut_all(groupdf):
                    for j in range(groupdf.shape[0]):
                        groupdf[i].iloc[j]=groupdf[i].iloc[j][0:len(kwargs[i])]
                        #print(len(groupdf[i].iloc[j][0:len(kwargs[i])]))
                    return groupdf
                newp=newp.groupby(i).apply(cut_all)
                newp=newp.groupby(i).get_group(kwargs[i])
            except KeyError:
                print("Not Found")
        return newp
    
    def cast_inner_extract(self,nums):
        cast_df=pd.DataFrame({"name":[],"gender":[],"id":[]},dtype=str)
        credits_df = self.credits_2_df.copy()
        n=0
        for _ in self.movies_df["cast"]:
            
            tempcredits=credits_df["cast"].iloc[n]
            tempcredits=ast.literal_eval(tempcredits)
            credits_cast=pd.DataFrame(tempcredits).filter(["name","gender","id"])
            #moviedata.search(credits_cast,gender=2) #2 male , 1 female, 0 none
            credits_cast.groupby("gender").filter(lambda x: x["gender"].iloc[0]>0) #kill zero ones
            print(self.credits_2_df.iloc[n]["id"])
            for i in range(credits_cast.shape[0]):
                
                
                credits_cast["id"].iloc[i]=str("["+str(self.credits_2_df.iloc[n]["id"])+"]")
                


            cast_df=pd.concat([cast_df,credits_cast],axis=0)
            #print(cast_df)
            #print (cast_df)
            n+=1
        return cast_df
        
        






In [665]:
dfcs=moviedata()


#dfcs.movies_df
#cast=dfcs.cast_inner_extract(nums=20)
moviedata.search(dfcs.movies_df,id="628")

,adult,popularity,budget,genres,homepage,id,imdb_id,original_title,overview,production_companies,production_countries,release_date,spoken_languages,vote_average,vote_count,cast,crew
17667,False,8.062578,40000000,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",http://www.colombiana-movie.com/,628,tt1657507,Colombiana,"Zoe Saldana plays a young woman who, after wit...","[{'name': 'TF1 Films Production', 'id': 3823},...","[{'iso_3166_1': 'FR', 'name': 'France'}]",2011-07-27,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",6.5,848,"[{'cast_id': 3, 'character': 'Cataleya Restrep...","[{'credit_id': '52fe468fc3a368484e09758f', 'de..."
18356,False,6.695229,37000000,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://dolphintalemovie.warnerbros.com/index.html,628,tt1564349,Dolphin Tale,A story centered on the friendship between a b...,"[{'name': 'Alcon Entertainment', 'id': 1088}, ...","[{'iso_3166_1': 'US', 'name': 'United States o...",2011-09-23,"[{'iso_639_1': 'en', 'name': 'English'}]",6.6,166,"[{'cast_id': 10, 'character': 'Dr. McCarthy', ...","[{'credit_id': '52fe468fc3a368484e097675', 'de..."
34262,False,1.824977,18000000,"[{'id': 18, 'name': 'Drama'}]",http://www.quartierlointain-lefilm.com/,628,tt1266037,Quartier Lointain,"Thomas, a father in his fifties, returns by ch...","[{'name': 'Entre Chien et Loup', 'id': 15472}]","[{'iso_3166_1': 'BE', 'name': 'Belgium'}, {'is...",2010-05-20,"[{'iso_639_1': 'fr', 'name': 'Français'}]",6.5,11,"[{'cast_id': 1, 'character': 'Sea Turtle (voic...","[{'credit_id': '52fe452a9251416c910299fd', 'de..."


In [623]:
credits_df = pd.read_csv("archive\credits-2.csv")
tempcredits=credits_df["cast"].iloc[0]
tempcredits=ast.literal_eval(tempcredits)
credits_cast=pd.DataFrame(tempcredits).filter(["name","character","gender"])
#moviedata.search(credits_cast,gender=2) #2 male , 1 female, 0 none
credits_cast.groupby("gender").filter(lambda x: x["gender"].iloc[0]>0) #kill zero ones
n=0
for row in credits_cast.duplicated(subset="gender"):
    if row:
        credits_cast.replace(credits_cast.iloc[n-1]["gender"],"lol")



In [ ]:
#周日：完成了df引入，movie和credit的df的合成但是没洗，待处理：
#   完成nah处理（洗）
#   完成cast处理，展开方法的书写
#   写出以人为分类方法的df
#   建立用户搜索方法